In [ ]:
#!pip install ipynb

In [ ]:
from ipynb.fs.full.fail_stop_agreement import agreement, Process
import threading

n = 4

In [ ]:
def test_same_input(input):
    threads = []
    processes = []

    for i in range(0, n):
        pr = Process(i, input)
        processes.append(pr)

        thr = threading.Thread(target=agreement, args=((pr,)))
        threads.append(thr)

    for thr in threads:    
        thr.start()

    for thr in threads:
        thr.join()

    print("*******   SOLUTION:   *******")
    for pr in processes:
        print("process(", pr.id, ") = ", pr.output, " @epoch: ", pr.decision_epoch)
        if pr.output != input or pr.decision_epoch != 1:
            return False
    print()
    return True

In [ ]:
test_same_input("0")
test_same_input("1")